In [61]:
import os
from os.path import dirname, abspath,join
import sys
import numpy as np

import torch

In [67]:
x = torch.tensor([[1,2,3,0,0,0,0],[1,2,3,0,0,0,1]])
dim = -1
seq_len = x.size(dim=dim)
zeros_number = seq_len-torch.count_nonzero(x,dim=dim).numpy()
if len(zeros_number) == 1:
    zeros_number = zeros_number.item()


torch.where(x==0,1,0)

tensor([[0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 0]])

In [56]:
zeros_number

array([0, 0, 0, 2, 2, 2, 1], dtype=int64)